## Network Performance Monitoring Infrastructure setup using 2 seperate VLANS connected to a each other via a router.

We use Chameleon DirectStitch ports to connect isolated tenant networks at each of the two Chameleon sites and Chicago and Texas.  


### Set Variables

  - `OS_PROJECT_NAME`: The name of the project you want to use.
  - `UC_RESERVATION`: Reservation ID at UC.
  - `TACC_RESERVATION`: Reservation ID at TACC.
  - `SSH_KEY_NAME`: SSH key name as registered in Chameleon
  - `SSH_PRIVATE_KEY`: Path to the corresponding private key in your Jupyter container.
  - `NETWORK_SUBNET`: Private IP subnet that will span both Chameleon sites.
  - `UC_ALLOCATION_START`: The start of the IP range assigned at the UC site.
  - `UC_ALLOCATION_END`: The end of the IP range assigned at the UC site.
  - `TACC_ALLOCATION_START`: The start of the IP range assigned at the TACC site.
  - `TACC_ALLOCATION_END`: The start of the IP range assigned at the TACC site.
  - `RESOURCE_PREFIX`: A begin string used to idenify resources provision. This includes your user ID so that your resources can be distinguised from those belonging to other tutorial particpants.
  - `GENI_PEM_FILE`: The path to your GENI certificate file in you Jupyter container.
  - `UC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at UC.
  - `TACC_DIRECTSTITCH_VLAN`: The VLAN using to attach to the DirectStitch port at TACC.

In [31]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='MC4N'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-uc-mc4n-key
export SSH_PRIVATE_KEY=${HOME}/work/bmohd-chameleon-jupyter

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-netperform-$(date +%b%d)"

export GENI_PEM_FILE="~/work/geni-bmohamme.pem"


In [30]:
pwd

/work/work


## Step 1: Create Chameleon Isolated Network(s) with DirectStitch Ports

Creating a Chameleon DirectStitch port is as easy as creating an isolated VLAN but with an extra parameter requesting the port.We create a two Chameleon networks each with a DirectConnect port. One network will be at the UC (Chicago) while the other will be at TACC (Austin).

### Step 1a: Create a Network a UC

#### Set network resource names

In [32]:
UC_RESERVATION_NAME=${RESOURCE_PREFIX}"-UC"

# Set the names for the network, subnet, router, and switch. 
UC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-UC"
UC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-UC"
UC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-UC"



#### Create the Network

In [33]:
OS_REGION_NAME='CHI@UC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+5 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=7,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${UC_NETWORK_NAME}",resource_properties='["==","$physical_network","exogeni"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${UC_RESERVATION_NAME}

UC_RESERVATION=`blazar lease-show --format value -c id ${UC_RESERVATION_NAME}`
echo UC_RESERVATION $UC_RESERVATION

UC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${UC_RESERVATION_NAME} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo UC_NODE_RESERVATION $UC_NODE_RESERVATION


Creating network
Matches: ('min=1,max=7', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '7')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=44b38c44-2a42-4b6d-b129-6c8f1b2a1375', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '44b38c44-2a42-4b6d-b129-6c8f1b2a1375')
Matches: (None, 'resource_type', 'virtual:floatingip')
Matches: ('resource_type=network,network_name=bmohd1-netperform-Jul02-Network-UC', 'resource_properties', '["==","$physical_network","exogeni"]')
Matches: ('resource_type=network', 'network_name', 'bmohd1-netperform-Jul02-Network-UC')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+------------------------------------------------------------------

#### Get the DirectStitch VLAN Tag

Note: this will not be successful until the network is created. You may need to rerun this command until the VLAN tag is found.

In [34]:
OS_REGION_NAME='CHI@UC'

UC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${UC_NETWORK_NAME}`
echo UC DirectStitch VLAN: $UC_DIRECTSTITCH_VLAN 

UC_FLOATING_IP=`lease_list_floating_ips $UC_RESERVATION`
echo UC_FLOATING_IP $UC_FLOATING_IP 


UC DirectStitch VLAN: 3295
UC_FLOATING_IP 192.5.87.186


#### Step 1b: Add a subnet and router to the network

In [35]:
OS_REGION_NAME='CHI@UC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${UC_ALLOCATION_START},end=${UC_ALLOCATION_END} \
                        --dhcp \
                        --network ${UC_NETWORK_NAME} \
                        ${UC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${UC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${UC_ROUTER_NAME} ${UC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${UC_ROUTER_NAME}

echo Network ${UC_NETWORK_NAME} is ready for nodes!

Creating Subnet
+----------------------+-------------------------------------------------------+
| Field                | Value                                                 |
+----------------------+-------------------------------------------------------+
| allocation_pools     | 192.168.100.10-192.168.100.19                         |
| cidr                 | 192.168.100.0/24                                      |
| created_at           | 2020-07-02T01:18:47Z                                  |
| description          |                                                       |
| dns_nameservers      |                                                       |
| dns_publish_fixed_ip | None                                                  |
| enable_dhcp          | True                                                  |
| gateway_ip           | 192.168.100.1                                         |
| host_routes          |                                                       |
| id        

#### Step 1c: Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [36]:
UC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@UC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-Ubuntu16.04" \
                        --key-name "uc-mc4n-key" \
                        --hint reservation=${UC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${UC_NETWORK_NAME} \
                        ${UC_NODE_NAME}

Creating servers... This will take several minutes!
+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         |                                        |
| OS-EXT-SRV-ATTR:host                | None                                   |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                   |
| OS-EXT-SRV-ATTR:instance_name       |                                        |
| OS-EXT-STS:power_state              | NOSTATE                                |
| OS-EXT-STS:task_state               | scheduling                             |
| OS-EXT-STS:vm_state                 | building                               |
| OS-SRV-USG:launched_at              | None             

In [37]:
OS_REGION_NAME='CHI@UC'
#Attach the floating IP address to the server
echo $UC_FLOATING_IP 
openstack server add floating ip $UC_NODE_NAME $UC_FLOATING_IP 

192.5.87.186


### Create Network a TACC

Repeat the steps but target the TACC Chameleon site

In [38]:
# Set up user's project (user's can be multiple ones, so there is no default currently)
export OS_PROJECT_NAME='MC4N'

# Set chameleon keypair name and path to the private ssh key
export SSH_KEY_NAME=${USERNAME}-tacc-mc4n-key
export SSH_PRIVATE_KEY=${HOME}/work/bmohd-chameleon-jupyter

export NETWORK_SUBNET="192.168.100.0/24"
export UC_ALLOCATION_START="192.168.100.10"
export UC_ALLOCATION_END="192.168.100.19"
export TACC_ALLOCATION_START="192.168.100.20"
export TACC_ALLOCATION_END="192.168.100.29"

export RESOURCE_PREFIX="$USER-netperform-$(date +%b%d)"

export GENI_PEM_FILE="~/work/geni-bmohamme.pem"


In [39]:
OS_REGION_NAME='CHI@TACC'

TACC_RESERVATION_NAME=${RESOURCE_PREFIX}"-TACC"

# Set the names for the network, subnet, router, and switch. 
# See above about using identifiable names.  
TACC_NETWORK_NAME=${RESOURCE_PREFIX}"-Network-TACC"
TACC_SUBNET_NAME=${RESOURCE_PREFIX}"-Subnet-TACC"
TACC_ROUTER_NAME=${RESOURCE_PREFIX}"-Router-TACC"

#### Create the Network

In [40]:
OS_REGION_NAME='CHI@TACC'

START_DATE=`date -d "+2 min" +'%F %H:%M'`
END_DATE=`date -d "+5 day" +'%F %H:%M'`

PUBLIC_NETWORK_ID=$(openstack network show public -c id -f value)

echo Creating network ${EXOGENI_STITCH_NAME}
blazar lease-create \
   --physical-reservation min=1,max=5,resource_properties='["=", "$node_type", "compute_haswell"]' \
   --reservation resource_type=virtual:floatingip,network_id=${PUBLIC_NETWORK_ID},amount=1 \
   --reservation resource_type="network",network_name="${TACC_NETWORK_NAME}",resource_properties='["==","$physical_network","exogeni"]' \
   --start-date "${START_DATE}" \
   --end-date "${END_DATE}" \
   ${TACC_RESERVATION_NAME}

TACC_RESERVATION=`blazar lease-show --format value -c id ${TACC_RESERVATION_NAME}`
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_RESERVATION=`blazar lease-show -json --format value -c reservations ${TACC_RESERVATION} | jq -r 'select(.resource_type | contains("physical:host")) | .id'`
echo TACC_NODE_RESERVATION $TACC_NODE_RESERVATION
   
   
   

Creating network
Matches: ('min=1,max=5', 'resource_properties', '["=", "$node_type", "compute_haswell"]')
Matches: ('min=1', 'max', '5')
Matches: (None, 'min', '1')
Matches: ('resource_type=virtual:floatingip,network_id=6189521e-06a0-4c43-b163-16cc11ce675b', 'amount', '1')
Matches: ('resource_type=virtual:floatingip', 'network_id', '6189521e-06a0-4c43-b163-16cc11ce675b')
Matches: (None, 'resource_type', 'virtual:floatingip')
Matches: ('resource_type=network,network_name=bmohd1-netperform-Jul02-Network-TACC', 'resource_properties', '["==","$physical_network","exogeni"]')
Matches: ('resource_type=network', 'network_name', 'bmohd1-netperform-Jul02-Network-TACC')
Matches: (None, 'resource_type', 'network')
Created a new lease:
+--------------+----------------------------------------------------------------------------+
| Field        | Value                                                                      |
+--------------+--------------------------------------------------------------

#### Get the DirectStitch VLAN Tag

Repeat until sucessful

In [41]:
OS_REGION_NAME='CHI@TACC'
TACC_DIRECTSTITCH_VLAN=`openstack network show -c provider:segmentation_id -f value ${TACC_NETWORK_NAME}`
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN 

TACC_FLOATING_IP=`lease_list_floating_ips $TACC_RESERVATION`
echo TACC_FLOATING_IP $TACC_FLOATING_IP 


TACC DirectStitch VLAN: 3509
TACC_FLOATING_IP 129.114.109.235


#### Add a subnet and router to the network

In [42]:
OS_REGION_NAME='CHI@TACC'
echo Creating Subnet
openstack subnet create --max-width 80 \
                        --subnet-range ${NETWORK_SUBNET} \
                        --allocation-pool start=${TACC_ALLOCATION_START},end=${TACC_ALLOCATION_END} \
                        --dhcp \
                        --network ${TACC_NETWORK_NAME} \
                        ${TACC_SUBNET_NAME}
                        
echo Creating Router
openstack router create --max-width 80 ${TACC_ROUTER_NAME}

echo Linking router to subnet
openstack router add subnet ${TACC_ROUTER_NAME} ${TACC_SUBNET_NAME}

echo Linking router to external gateway
openstack router set --external-gateway public ${TACC_ROUTER_NAME}

echo Network ${TACC_NETWORK_NAME} is ready for nodes!

Creating Subnet
+----------------------+-------------------------------------------------------+
| Field                | Value                                                 |
+----------------------+-------------------------------------------------------+
| allocation_pools     | 192.168.100.20-192.168.100.29                         |
| cidr                 | 192.168.100.0/24                                      |
| created_at           | 2020-07-02T01:31:02Z                                  |
| description          |                                                       |
| dns_nameservers      |                                                       |
| dns_publish_fixed_ip | None                                                  |
| enable_dhcp          | True                                                  |
| gateway_ip           | 192.168.100.1                                         |
| host_routes          |                                                       |
| id        

#### Launch servers connected to the new network

At this point your OpenFlow network is ready for compute nodes. You can add nodes using the CLI commands below or by any other method you are comfortable with. 

In [43]:
echo TACC_RESERVATION $TACC_RESERVATION

TACC_NODE_NAME=${RESOURCE_PREFIX}-node

OS_REGION_NAME='CHI@TACC'
echo Creating servers... This will take several minutes! 
openstack server create --max-width 80 \
                        --flavor "baremetal" \
                        --image "CC-Ubuntu16.04"  \
                        --key-name "tacc-mc4n-key" \
                        --hint reservation=${TACC_NODE_RESERVATION} \
                        --security-group default  \
                        --nic net-id=${TACC_NETWORK_NAME} \
                        ${TACC_NODE_NAME}


TACC_RESERVATION 4087c400-e8c4-4b33-8029-311554e22103
Creating servers... This will take several minutes!
+-------------------------------------+----------------------------------------+
| Field                               | Value                                  |
+-------------------------------------+----------------------------------------+
| OS-DCF:diskConfig                   | MANUAL                                 |
| OS-EXT-AZ:availability_zone         |                                        |
| OS-EXT-SRV-ATTR:host                | None                                   |
| OS-EXT-SRV-ATTR:hypervisor_hostname | None                                   |
| OS-EXT-SRV-ATTR:instance_name       |                                        |
| OS-EXT-STS:power_state              | NOSTATE                                |
| OS-EXT-STS:task_state               | scheduling                             |
| OS-EXT-STS:vm_state                 | building                               |
| O

In [47]:
OS_REGION_NAME='CHI@TACC'

#Attach the floating IP address to the server
echo $TACC_FLOATING_IP 
openstack server add floating ip $TACC_NODE_NAME $TACC_FLOATING_IP 

129.114.109.235


## Step 2: Use ExoGENI to Connect the Ports


In [ ]:
echo UC DirectStitch VLAN:   $UC_DIRECTSTITCH_VLAN
echo TACC DirectStitch VLAN: $TACC_DIRECTSTITCH_VLAN
xoStitch create -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE

## Experiment with the DirectStitch Connection

Add a floating IP to the nodes and ssh to them in separate windows.

- Ping from one node to the other using the fixed IPs. What is the RTT?
- Use traceroute find the path between the nodes. How many hops do you see if you use the fixed IPs? How many hops uing the floating IP? 
- Use iperf3 to test the bandwidth between the nodes. Set the MTU to use jumbo frames. Does the bandwidth change?

## Other Useful Commands

In [ ]:
#Get the status of the ExoGENI circuit
xoStitch status -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE


In [ ]:
#Delete the ExoGENI circuit
xoStitch delete -sp1 uc -vlan1 $UC_DIRECTSTITCH_VLAN -sp2 tacc -vlan2 $TACC_DIRECTSTITCH_VLAN -c $GENI_PEM_FILE
